In [1]:
import numpy as np

import os

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

from sklearn.utils import class_weight

from load_moonboard import load_moonboard

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [16]:
class MoonBoardDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train = True):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train = train
        
        (self.x_train, self.y_train), (self.x_test, self.y_test) = load_moonboard()
        self.x_train = self.x_train.transpose(0,1,3,2 ).astype(float)
        self.x_test = self.x_test.transpose(0,1,3,2 ).astype(float)
        self.y_train = self.y_train.reshape(-1,1).astype(int)
        self.y_test = self.y_test.reshape(-1,1).astype(int)
        #self.y_train = np.eye(17)[self.y_train]
        #self.y_test = np.eye(17)[self.y_test]

    def __len__(self):
        if self.train:
            return len(self.x_train)
        else:
            return len(self.x_test)

    def __getitem__(self, idx):
        if self.train:
            return self.x_train[idx], self.y_train[idx]
        else:
            return self.x_test[idx], self.y_test[idx]


    # prepare data

dataset = MoonBoardDataset(train = True)

train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=128, shuffle=True)
    
test_loader = torch.utils.data.DataLoader(
    MoonBoardDataset(train = False),
    batch_size=32, shuffle=True)

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 2, 1)
        self.conv2 = nn.Conv2d(20, 50, 2, 1)
        self.fc1 = nn.Linear(3*11*18, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 500)
        self.fc4 = nn.Linear(500, 500)
        self.fc5 = nn.Linear(500, 500)
        self.fc6 = nn.Linear(500, 500)
        self.fc7 = nn.Linear(500, 500)
        self.fc8 = nn.Linear(500, 13)

    def forward(self, x):
        x = x.view(-1, 3*11*18)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        #x = self.fc5(x)
        #x = self.fc6(x)
        #x = self.fc7(x)
        x = self.fc8(x)
        return x
    
    
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.plans = 128
        self.conv1 = nn.Conv2d(3, self.plans, 1)
        self.conv2 = nn.Conv2d(self.plans, self.plans*2, 5, 1, 3)
        self.conv3 = nn.Conv2d(self.plans*2, self.plans*4, 5, 1, 3)
        self.relu =  nn.ReLU(True)
        self.pool = nn.MaxPool2d(2)
        self.drop = nn.Dropout2d(0.2)
        self.fc1 = nn.Linear(self.plans*4*6*4, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 13)
        self.softmax = nn.Softmax()
        
        self.bn1 = nn.BatchNorm2d(self.plans)
        self.bn2 = nn.BatchNorm2d(self.plans*2)
        self.bn3 = nn.BatchNorm2d(self.plans*4)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        #x = self.pool(x)
        x = self.relu(x)
        x = self.drop(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.pool(x)
        x = x.view(-1, self.plans*4*6*4)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.fc3(x)
        #x = nn.functional.log_softmax(x, dim= 1)
        return x
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=16):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        
        self.softmax = nn.Softmax(dim = 1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        #out = self.softmax(out)
        return out
    
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [56]:
torch.manual_seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


model = ResNet(BasicBlock, [1,1,1,1]).to(device)
#model = ConvNet().to(device)

cuda


In [57]:
optimizer = optim.Adam(model.parameters(), weight_decay=0.01)
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.7)

In [73]:
def acc(model, test=True):
    correct = 0
    total = 0
    with torch.no_grad():
        if test:
            loader = test_loader
        else:
            loader = train_loader
        for data in loader:
            images, labels = data
            outputs = model(images.to(device, dtype=torch.float32))

            outputs = outputs.view((-1,16))   
            _, predicted = torch.max(outputs.data, 1)

            
            labels = labels.view((-1))
            total += labels.size(0)
            correct += (abs(predicted - labels.to(device, dtype=torch.long))<1).sum().item()

    if test:
        st = "test"
    else:
        st = "train"
    print('Accuracy of the network on ',st,' images: %f' % (
        100. * float(correct) / float(total)))
    return 100. * float(correct) / float(total)
    
    
def acc_class(model):
    class_correct = list(0. for i in range(18))
    class_total = list(0. for i in range(18))
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images.to(device, dtype=torch.float32))
            outputs = outputs.view((-1,16))

            _, predicted = torch.max(outputs.data, 1)
            

            labels = labels.view((-1))
            c = (predicted == labels.to(device, dtype=torch.long))
            for i in range(len(labels)):
                label = labels[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(0,18):
        if class_total[i] != 0:
            print('Accuracy of %5s : %2d %%' % (
                i, 100. * class_correct[i] // class_total[i]))
    

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device, dtype=torch.float32), target.to(device, dtype=torch.long)
        optimizer.zero_grad()
        output = model(data)
        target = target.view((-1))
        loss = F.cross_entropy(output, target)#, weight=torch.tensor(weights).to(device, dtype = torch.float))
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx // len(train_loader), loss.item()))
    
        



In [74]:
best = 0

In [75]:
for i in range(30):
    train(model, device, train_loader, optimizer, i)
    ac = acc(model)
    acc(model, test= False)
    if ac > best:
        best = ac
        torch.save(model,"inception_model.trch")
    
acc_class(model)



Train Epoch: 0 [0/4129 (0%)]	Loss: 1.104287
Train Epoch: 0 [1280/4129 (30%)]	Loss: 0.330170
Train Epoch: 0 [2560/4129 (60%)]	Loss: 0.189032
Train Epoch: 0 [3840/4129 (90%)]	Loss: 0.192738
Accuracy of the network on  test  images: 89.157793
Accuracy of the network on  train  images: 96.657786
Train Epoch: 1 [0/4129 (0%)]	Loss: 0.156609
Train Epoch: 1 [1280/4129 (30%)]	Loss: 0.153735
Train Epoch: 1 [2560/4129 (60%)]	Loss: 0.077351
Train Epoch: 1 [3840/4129 (90%)]	Loss: 0.116990
Accuracy of the network on  test  images: 90.416263
Accuracy of the network on  train  images: 98.813272
Train Epoch: 2 [0/4129 (0%)]	Loss: 0.049157
Train Epoch: 2 [1280/4129 (30%)]	Loss: 0.102245
Train Epoch: 2 [2560/4129 (60%)]	Loss: 0.024890
Train Epoch: 2 [3840/4129 (90%)]	Loss: 0.022466
Accuracy of the network on  test  images: 91.674734
Accuracy of the network on  train  images: 99.515621
Train Epoch: 3 [0/4129 (0%)]	Loss: 0.074565
Train Epoch: 3 [1280/4129 (30%)]	Loss: 0.026148
Train Epoch: 3 [2560/4129 (60

Accuracy of the network on  train  images: 98.232017
Train Epoch: 28 [0/4129 (0%)]	Loss: 0.087750
Train Epoch: 28 [1280/4129 (30%)]	Loss: 0.074251
Train Epoch: 28 [2560/4129 (60%)]	Loss: 0.110028
Train Epoch: 28 [3840/4129 (90%)]	Loss: 0.143294
Accuracy of the network on  test  images: 90.029042
Accuracy of the network on  train  images: 98.038266
Train Epoch: 29 [0/4129 (0%)]	Loss: 0.069195
Train Epoch: 29 [1280/4129 (30%)]	Loss: 0.119528
Train Epoch: 29 [2560/4129 (60%)]	Loss: 0.072671
Train Epoch: 29 [3840/4129 (90%)]	Loss: 0.050498
Accuracy of the network on  test  images: 91.093901
Accuracy of the network on  train  images: 99.224994
Accuracy of     0 : 94 %
Accuracy of     1 : 73 %
Accuracy of     2 : 48 %


In [34]:
(x_train, y_train), (x_test, y_test) = load_moonboard()


In [69]:
count_train = [0]*16
for label in y_train:
    count_train[label] +=1
print(count_train)
s = sum(count_train)
weights = [s**6/(i*(1000**5))-1300  if i != 0 else 0 for i in count_train]
print(weights)

count_test = [0]*16
for label in y_test:
    count_test[label] +=1
print(count_test)

print(sum(count_test) + sum(count_train))

[3346, 608, 175, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[180.96010919693754, 6850.152179889726, 27015.957287845446, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[845, 144, 44, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5162
